# Automated Hamiltonian Extraction


## Imports and file structure creation

In [28]:
import os
from qiskit_metal import MetalGUI
from scipy.constants import Planck, e, hbar, h
import numpy as np

from validate_config_file import *
from helpers import *
from make_designs import *
from prepare_inductex_gds_files import *
from prepare_palace_sims import *
from prepare_inductex_sims import *
from run_simulations import *
from collect_simulation_results import *
from quantum_analysis_lom import *
from quantum_analysis_epr import *
from generate_report import *
from analyse_hamiltonian import *
from post_processing import *

# MPI related environment variables 
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ["PMIX_MCA_gds"]="hash"

gds_types                       = ['raw', 'processed_multi_layer', 'processed_single_layer']
design_names                    = ['full', 'full_no_jj', 'junction', 'transmon', 'transmon_no_jj', 'resonator', 'feedline']
inductex_simulation_types       = ['capacitance', 'inductance', 'sparams', 'electric_field', 'magnetic_field']
palace_simulation_types         = ['capacitance', 'eigenmode']
os.makedirs('data', exist_ok=True)
for gds_type in gds_types:
    os.makedirs(f'gds_files/{gds_type}', exist_ok=True)

for design in design_names:
    for sim_type in inductex_simulation_types:
        os.makedirs(f'simulations/inductex/{sim_type}/{design}', exist_ok=True)


for design in design_names:
    for sim_type in palace_simulation_types:
        os.makedirs(f'simulations/palace/{sim_type}/{design}', exist_ok=True)


# TODO
# Add suport for none entries in config file
# Double check chip z height is correct
# Find a way to remove the xmon dummy
#   Prehaps this can bedone with replace junction
# Cell hirachy might be a bit busted for GDS files
# Multiple declarations of component geos, check helper functions for uncertainty
# claw_cpw_width and calw_cpw_length dont seem to do anything
# The json file for all the geometry data is slightly contradictory as geometry for some things have been declared 2 or 3 times
# this is most likely due to different simulations being done on indvidiual components or a partial design of the system
# Removed end_straight=resonator_geo['end_straight'] from resonator as it was none and causing issues
# Bad implementation of pin_width = pin_width*1e-3 in _get_pin_left_um and _get_pin_right_um
# Force adjusted CPW for transmon to fit full design, didnt work use in report for source of error
# Sparam for transmon is not quite right, will fix later
# Sparams are pasted in weirdly
# Slight bug with palace saying succesful or unsucessful when keep_open is true
# Way too much effort to get transmon eigen sim working, don't need it since we can just use the full, will just cause problems
# Using cavity-qubit notation in LOM analysis
# if empty results continue
# Structure name of control file is off
# Weird problem with making sims if using palace for certain designs

## Load and validate setup file

In [29]:
CONFIG_PATH                     = "quantum_config.json"
quantum_config                  = ensure_config(CONFIG_PATH)
run_inductex                    = False
run_palace_setup                = False
run_palace                      = False
palace_show_conductor_indices   = False

[config] Validation successful: 'quantum_config.json' is valid.
[INFO] design_specs"]["qubit_frequency_ghz is: 4.1 (Default: 3.7)
[INFO] design_specs"]["cavity_frequency_ghz is: 7.5 (Default: 6.98)
[INFO] design_specs"]["anharmonicity_mhz is: -180 (Default: -210)
[INFO] design_specs"]["coupling_g_mhz is: 120 (Default: 100)
[INFO] design_specs"]["kappa_khz is: 500 (Default: 405)
[INFO] simulations"]["palace"]["eigenmode"]["port_1_resistance is: 50 (Default: 45)


## Find design from database and get master design file


In [30]:
design_specs                                 = quantum_config["design_specs"]

if os.path.exists("data/design_reference_data_all.json"):
    best_design, ref_design_specs, ref_cap_data, qubit_geo, resonator_geo, feedline_geo, junction_inductance = fetch_design_fast()
else:
    best_design, ref_design_specs, ref_cap_data, qubit_geo, resonator_geo, feedline_geo, junction_inductance = fetch_design(design_specs=design_specs, num_top=1)

## Save master design file and useful data

In [31]:
save_results_to_json(best_design, name="data/design_reference_data_all", save=True)
save_results_to_json(ref_design_specs, name="data/design_reference_data_design_specs", save=True)
save_results_to_json(ref_cap_data, name="data/design_reference_data_capacitance", save=True)
save_results_to_json(qubit_geo, name="data/design_reference_qubit_geometry", save=True)
save_results_to_json(resonator_geo, name="data/design_reference_resonator_geometry", save=True)
save_results_to_json(feedline_geo, name="data/design_reference_feedline_geometry", save=True)
save_results_to_json(junction_inductance, name="data/design_reference_junction_inductance", save=True)

✅ Saved to data/design_reference_data_all.json
✅ Saved to data/design_reference_data_design_specs.json
✅ Saved to data/design_reference_data_capacitance.json
✅ Saved to data/design_reference_qubit_geometry.json
✅ Saved to data/design_reference_resonator_geometry.json
✅ Saved to data/design_reference_feedline_geometry.json
✅ Saved to data/design_reference_junction_inductance.json


## Generate Qiskit Metal designs from master design geomtery and config file

In [32]:
scaling_factor              = quantum_config["miscellaneous"]["sims"]["base_chip_scaling_factor"]
full_design                 = make_full_design(best_design, qubit_geo, resonator_geo, feedline_geo)
full_no_jj_design           = make_full_no_jj_design(best_design, qubit_geo, resonator_geo, feedline_geo)
junction_design             = make_junction_design(best_design, qubit_geo, resonator_geo, feedline_geo, scaling_factor=scaling_factor)
transmon_design             = make_transmon_design(best_design, qubit_geo, resonator_geo, feedline_geo, scaling_factor=scaling_factor)
transmon_no_jj_design       = make_transmon_no_jj_design(best_design, qubit_geo, resonator_geo, feedline_geo, scaling_factor=scaling_factor)
resonator_design            = make_resonator_design(best_design, qubit_geo, resonator_geo, feedline_geo, scaling_factor=scaling_factor)
feedline_design             = make_feedline_design(best_design, qubit_geo, resonator_geo, feedline_geo, scaling_factor=scaling_factor)

## Save raw designs to GDS

In [33]:
to_gds(full_design, filename="gds_files/raw/full.gds")  
to_gds(full_no_jj_design, filename="gds_files/raw/full_no_jj.gds")  
to_gds(junction_design, filename="gds_files/raw/junction.gds")  
to_gds(transmon_design, filename="gds_files/raw/transmon.gds")  
to_gds(transmon_no_jj_design, filename="gds_files/raw/transmon_no_jj.gds")  
to_gds(resonator_design, filename="gds_files/raw/resonator.gds")  
to_gds(feedline_design, filename="gds_files/raw/feedline.gds")  

04:54PM 34s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
04:54PM 34s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"c:\Users\reece\Desktop\Final Year Work\2nd Semester\Project (E) 448\Simulations\resources".
04:54PM 35s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"c:\Users\reece\Desktop\Final Year Work\2nd Semester\Project (E) 448\Simulations\resources".
04:54PM 35s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
04:54PM 35s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to rep

## Process InductEx GDS files

In [34]:
process_full_gds()
process_full_no_jj_gds()
process_junction_gds()
process_transmon_gds()
process_transmon_no_jj_gds()
process_resonator_gds()
process_feedline_gds()

Boolean AND((1, 0), (3, 0)) → (0, 0) complete.
Boolean AND((0, 0), (4, 0)) → (0, 0) complete.
Boolean OR((4, 10), (4, 11)) → (4, 10) complete.
Deleted layer 1, datatype 0
Deleted layer 3, datatype 0
Deleted layer 4, datatype 0
Deleted layer 4, datatype 11
✅  Wrote gds_files/processed_multi_layer/full_processed_multi.gds
Boolean OR((0, 0), (1, 0)) → (0, 0) complete.
Boolean OR((0, 0), (2, 0)) → (0, 0) complete.
Boolean OR((0, 0), (3, 0)) → (0, 0) complete.
Boolean OR((0, 0), (4, 0)) → (0, 0) complete.
Deleted layer 1, datatype 0
Deleted layer 2, datatype 0
Deleted layer 3, datatype 0
Deleted layer 4, datatype 0
✅  Wrote gds_files/processed_single_layer/full_processed.gds
Deleted layer 2, datatype 0
✅  Wrote gds_files/processed_multi_layer/full_no_jj_processed_multi.gds
Boolean OR((0, 0), (1, 0)) → (0, 0) complete.
Boolean OR((0, 0), (3, 0)) → (0, 0) complete.
Boolean OR((0, 0), (4, 0)) → (0, 0) complete.
Deleted layer 1, datatype 0
Deleted layer 3, datatype 0
Deleted layer 4, datatype 0

## Setup InductEx GDS simulation files

In [35]:
# Capacitance Sim GDS Files
make_inductex_cap_sim_transmon_gds(transmon_design, qubit_geo)              
make_inductex_cap_sim_transmon_no_jj_gds(transmon_no_jj_design, qubit_geo)  
make_inductex_cap_sim_resonator_gds(resonator_design, resonator_geo)        
make_inductex_cap_sim_feedline_gds(feedline_design, feedline_geo)           

# Inductance Sim GDS Files
make_inductex_ind_sim_resonator_gds(resonator_design, resonator_geo)        

# S-Parameter Sim GDS Files
make_inductex_sparam_sim_transmon_gds(transmon_design, qubit_geo)                   
make_inductex_sparam_sim_transmon_no_jj_gds(transmon_no_jj_design, qubit_geo)       
make_inductex_sparam_sim_resonator_gds(resonator_design, resonator_geo)             

✅  Wrote simulations/inductex/capacitance/transmon/transmon_cap.gds
✅  Wrote simulations/inductex/capacitance/transmon_no_jj/transmon_no_jj_cap.gds
✅  Wrote simulations/inductex/capacitance/resonator/resonator_cap.gds
✅  Wrote simulations/inductex/capacitance/feedline/feedline_cap.gds
✅  Wrote simulations/inductex/inductance/resonator/resonator_ind.gds
✅  Wrote simulations/inductex/sparams/transmon/transmon_sparams.gds
✅  Wrote simulations/inductex/sparams/transmon_no_jj/transmon_no_jj_sparams.gds
✅  Wrote simulations/inductex/sparams/resonator/resonator_sparams.gds


In [36]:
# UNIMPLEMENTED INDUCTEX GDS SIM FILES

#make_inductex_cap_sim_full_gds(full_design, qubit_geo)
#make_inductex_cap_sim_full_no_jj_gds(full_no_jj_design, qubit_geo)
#make_inductex_cap_sim_junction_gds(junction_design, qubit_geo)

#make_inductex_ind_sim_full_gds(full_design, qubit_geo)
#make_inductex_ind_sim_full_no_jj_gds(full_no_jj_design, qubit_geo)
#make_inductex_ind_sim_junction_gds(junction_design, qubit_geo)
#make_inductex_ind_sim_transmon_gds(transmon_design, qubit_geo)
#make_inductex_ind_sim_transmon_no_jj_gds(transmon_no_jj_design, qubit_geo)
#make_inductex_ind_sim_feedline_gds(feedline_design, feedline_geo)

#make_inductex_sparam_sim_full_gds(full_design, qubit_geo)
#make_inductex_sparam_sim_full_no_jj_gds(full_no_jj_design, qubit_geo)
#make_inductex_sparam_sim_junction_gds(junction_design, qubit_geo)
#make_inductex_sparam_sim_feedline_gds(feedline_design, feedline_geo)

## Setup InductEx IXI, LDF and CIR simulation files

In [37]:
quantum_config                    = ensure_config(CONFIG_PATH)   # Load config again for on the fly changes

inductex_ixi_config               = quantum_config["simulations"]["inductex"]["ixi"]
inductex_ldf_config               = quantum_config["simulations"]["inductex"]["layer_definition_file"]

# Generate InductEx simulation GDS files for all designs
designs = [full_design, full_no_jj_design, junction_design, transmon_design, transmon_no_jj_design, resonator_design, feedline_design]
for design_name, design_obj in zip(design_names, designs):
    make_inductex_cap_sim(design=design_obj, 
                          design_name=design_name, 
                          create_mask=False, 
                          mask_component=design_name, 
                          inductex_ixi_config=inductex_ixi_config, 
                          inductex_ldf_config=inductex_ldf_config
                          )

    make_inductex_ind_sim(design=design_obj, 
                          design_name=design_name, 
                          create_mask=False, 
                          mask_component=design_name, 
                          inductex_ixi_config=inductex_ixi_config, 
                          inductex_ldf_config=inductex_ldf_config
                          )

    make_inductex_sparam_sim(design=design_obj, 
                             design_name=design_name, 
                             create_mask=False, 
                             mask_component=design_name, 
                             inductex_ixi_config=inductex_ixi_config, 
                             inductex_ldf_config=inductex_ldf_config
                             )


[config] Validation successful: 'quantum_config.json' is valid.
[INFO] design_specs"]["qubit_frequency_ghz is: 4.1 (Default: 3.7)
[INFO] design_specs"]["cavity_frequency_ghz is: 7.5 (Default: 6.98)
[INFO] design_specs"]["anharmonicity_mhz is: -180 (Default: -210)
[INFO] design_specs"]["coupling_g_mhz is: 120 (Default: 100)
[INFO] design_specs"]["kappa_khz is: 500 (Default: 405)
[INFO] simulations"]["palace"]["eigenmode"]["port_1_resistance is: 50 (Default: 45)


## Setup PALACE Mesh Simulation Files 

In [38]:
if run_palace_setup:
    quantum_config                               = ensure_config(CONFIG_PATH)   # Load config here for easy use

    palace_cap_config               = quantum_config["simulations"]["palace"]["capacitance"]
    palace_eigenmode_config         = quantum_config["simulations"]["palace"]["eigenmode"]
    palace_eigenmode_config["lj"]   = junction_inductance

    cap_sim_list = []
    cap_sim_list.append(make_palace_cap_sim(transmon_design, "transmon", palace_cap_config))        
    cap_sim_list.append(make_palace_cap_sim(transmon_no_jj_design, "transmon_no_jj", palace_cap_config))
    cap_sim_list.append(make_palace_cap_sim(resonator_design, "resonator", palace_cap_config))        
    cap_sim_list.append(make_palace_cap_sim(feedline_design, "feedline", palace_cap_config))          

    eigen_sim_list = []
    eigen_sim_list.append(make_palace_eigenmode_sim(full_design, "full", palace_eigenmode_config))   
    #eigen_sim_list.append(make_palace_eigenmode_sim(full_no_jj_design, "full_no_jj", palace_eigenmode_config))      # Tested this, same result as full design so don't need to run
    eigen_sim_list.append(make_palace_eigenmode_sim(resonator_design, "resonator", palace_eigenmode_config))        # Impedance matching issue, can get result but result is off


In [39]:
# UNIMPLEMENTED PALACE MESH SIM FILES

#make_palace_cap_sim_full_gds(full_design, qubit_geo)
#make_palace_cap_sim_full_no_jj_gds(full_no_jj_design, qubit_geo)
#make_palace_cap_sim_junction_gds(junction_design, qubit_geo)

#make_palace_eigenmode_sim(junction_design, "junction", palace_eigenmode_config)
#make_palace_eigenmode_sim(feedline_design, "feedline", palace_eigenmode_config)
#make_palace_eigenmode_sim(transmon_design, "transmon", palace_eigenmode_config)   
#make_palace_eigenmode_sim(transmon_no_jj_design, "transmon_no_jj", palace_eigenmode_config)

## Optionally show sims and their conductor indices

In [40]:
if palace_show_conductor_indices:
    for cap_sim in cap_sim_list:
        cap_sim.display_conductor_indices()
    for eigen_sim in eigen_sim_list:
        #epr_data = eigen_sim.retrieve_EPR_data()  
        #mode_port_epr = eigen_sim.retrieve_mode_port_EPR()
        pass

## Run Inductex simulations

In [41]:
# Select either capacitance, inductance, sparams, or eigenmode 
if run_inductex:
    run_inductex_simulation("transmon", "capacitance", keep_open=False)
    run_inductex_simulation("transmon_no_jj", "capacitance", keep_open=False)
    run_inductex_simulation("resonator", "capacitance", keep_open=False)
    run_inductex_simulation("feedline", "capacitance", keep_open=False)        

    run_inductex_simulation("resonator", "inductance", keep_open=False)

    #run_inductex_simulation("transmon", "sparams", keep_open=False)
    #run_inductex_simulation("transmon_no_jj", "sparams", keep_open=False)
    #run_inductex_simulation("resonator", "sparams", keep_open=False)


## Run PALACE Simulations

In [42]:
if run_palace:
    palace_cap_config               = quantum_config["simulations"]["palace"]["capacitance"]
    palace_eigenmode_config         = quantum_config["simulations"]["palace"]["eigenmode"]
    palace_eigenmode_config["lj"]   = junction_inductance
    run_palace_simulation("transmon", "capacitance", palace_eigenmode_config["number_of_cores"], keep_open=False)
    run_palace_simulation("transmon_no_jj", "capacitance", palace_eigenmode_config["number_of_cores"], keep_open=False)
    run_palace_simulation("resonator", "capacitance", palace_eigenmode_config["number_of_cores"], keep_open=False)
    run_palace_simulation("feedline", "capacitance", palace_eigenmode_config["number_of_cores"], keep_open=False)

    run_palace_simulation("full", "eigenmode", palace_eigenmode_config["number_of_cores"], keep_open=False)
    #run_palace_simulation("full_no_jj", "eigenmode", palace_eigenmode_config["number_of_cores"], keep_open=False)
    run_palace_simulation("resonator", "eigenmode", palace_eigenmode_config["number_of_cores"], keep_open=False)


## Extract and save results from simulations

In [43]:
results_inductex, times_inductex = collect_inductex_results(design_names, inductex_simulation_types)
results_palace, times_palace    = collect_palace_results(design_names, palace_simulation_types)

save_results_to_json(results_inductex, name="data/inductex_sim_results", save=True, orient="records", indent=2)
save_results_to_json(results_palace, name="data/palace_sim_results", save=True, orient="records", indent=2)
save_results_to_json(times_inductex, name="data/inductex_sim_times", save=True, orient="records", indent=2)
save_results_to_json(times_palace, name="data/palace_sim_times", save=True, orient="records", indent=2)

✅ Saved to data/inductex_sim_results.json
✅ Saved to data/palace_sim_results.json
✅ Saved to data/inductex_sim_times.json
✅ Saved to data/palace_sim_times.json


## Post process results from simulations

In [44]:
other_computed_results = compute_other_values(ref_design_specs, ref_cap_data, results_inductex, results_palace)

## Quantum Analysis: Local Oscillator Method (LOM)

In [45]:
Nq                      = quantum_config["quantum_analysis"]["lom"]["qubit_truncation"]
Nc                      = quantum_config["quantum_analysis"]["lom"]["cavity_truncation"]
Nmax                    = quantum_config["quantum_analysis"]["lom"]["charge_truncation"]

## LOM Hamiltonian using HFSS reference results

In [46]:
Ec                      = ref_cap_data["EC"] * 1e9 * h  # Convert GHz to Hz and then to J
Cs_hfss                 = ref_cap_data["cross_to_cross"] * 1e-15  # Convert fF to F
Cg_hfss                 = ref_cap_data["cross_to_claw"] * 1e-15  # Convert fF to F
Cj_hfss                 = other_computed_results["cj_hfss"]  # Cj ~ 0 fF 
Lj_hfss                 = junction_inductance
Lr_hfss                 = results_inductex["inductance"]["resonator"]["L1"]  # Inductance not given so use inductex result
Cr_hfss                 = other_computed_results["cr_hfss"]
hamiltonian_lom_hfss    = compute_hamiltonian_lom(Lj=Lj_hfss, Lr=Lr_hfss, Cj=Cj_hfss, Cs=Cs_hfss, Cg=Cg_hfss, Cr=Cr_hfss, Nc=Nc, Nq=Nq, Nmax=Nmax)


## LOM Hamiltonian using InductEx Results 

In [47]:
Cs_inductex                  = results_inductex["capacitance"]["transmon_no_jj"]["CCROSS-CCROSS"]
Cg_inductex                  = results_inductex["capacitance"]["transmon_no_jj"]["CCROSS-CCLAW"]
Cj_inductex                  = 0  
Lj_inductex                  = junction_inductance
Lr_inductex                  = results_inductex["inductance"]["resonator"]["L1"]  # Inductance not given so use InductEx result
Cr_inductex                  = results_inductex["capacitance"]["resonator"]["CRESONATOR-CRESONATOR"]
hamiltonian_lom_inductex     = compute_hamiltonian_lom(Lj=Lj_inductex, Lr=Lr_inductex, Cj=Cj_inductex, Cs=Cs_inductex, Cg=Cg_inductex, Cr=Cr_inductex, Nc=Nc, Nq=Nq, Nmax=Nmax)

## LOM Hamiltonian using PALACE Results 

In [48]:
Cs_palace                  = results_palace["capacitance"]["transmon_no_jj"]["C2-C2"]
Cg_palace                  = results_palace["capacitance"]["transmon_no_jj"]["C2-C3"]
Cj_palace                  = 0                                                              
Lj_palace                  = junction_inductance
Cr_palace                  = results_palace["capacitance"]["resonator"]["C2-C2"]
Lr_extracted_palace        = other_computed_results["lr_palace"]
hamiltonian_lom_palace     = compute_hamiltonian_lom(Lj=Lj_palace, Lr=Lr_extracted_palace, Cj=Cj_palace, Cs=Cs_palace, Cg=Cg_palace, Cr=Cr_palace, Nc=Nc, Nq=Nq, Nmax=Nmax)

## Quantum Analysis: Energy Participation Ratio Method (EPR)

## EPR Hamiltonian using InductEx Results 

In [49]:
hamiltonian_epr_inductex = 0

## EPR Hamiltonian using PALACE Results 

In [50]:
mode_alt_cavity = "mode_2"

Lj                      = junction_inductance
Fq_Hz                   = results_palace["eigenmode"]["full"]["eigen_mode_frequencies"]["mode_1"]*1e9
Pq                      = abs(results_palace["eigenmode"]["full"]["eigen_mode_epr"]["mode_1"])
Fc_Hz                   = results_palace["eigenmode"]["full"]["eigen_mode_frequencies"][mode_alt_cavity]*1e9               # Use full design not resonator
Pc                      = abs(results_palace["eigenmode"]["full"]["eigen_mode_epr"][mode_alt_cavity])
N                       = quantum_config["quantum_analysis"]["epr"]["mode_truncation"]
hamiltonian_epr_palace  = compute_hamiltonian_epr(Lj_H=Lj, p_c=Pc, p_q=Pq, f_c_Hz=Fc_Hz, f_q_Hz=Fq_Hz, N=N)

## Extract Charactertics from Hamiltonians 

In [51]:
results_lom_hfss            = analyse_hamilotonian(hamiltonian_lom_hfss)
results_lom_inductex        = analyse_hamilotonian(hamiltonian_lom_inductex)
results_lom_palace          = analyse_hamilotonian(hamiltonian_lom_palace)
results_epr_inductex        = analyse_hamilotonian(hamiltonian_epr_inductex)
results_epr_palace          = analyse_hamilotonian(hamiltonian_epr_palace)

Hamiltonian is Hermitian
Is diagonal? True
Reconstruction ok? True
Hamiltonian is Hermitian
Is diagonal? True
Reconstruction ok? True
Hamiltonian is Hermitian
Is diagonal? True
Reconstruction ok? True
Hamiltonian is not an array (type: <class 'int'>) - returning zero results
Hamiltonian is Hermitian
Is diagonal? True
Reconstruction ok? True


In [52]:
generate_quantum_report(Lj, 
                        quantum_config, 
                        best_design, 
                        results_inductex, 
                        results_palace, 
                        hamiltonian_lom_hfss, 
                        hamiltonian_lom_inductex,
                        hamiltonian_lom_palace,
                        hamiltonian_epr_inductex,
                        hamiltonian_epr_palace,
                        results_lom_hfss,
                        results_lom_inductex,
                        results_lom_palace,
                        results_epr_inductex,
                        results_epr_palace,
                        other_computed_results,
                        times_inductex=times_inductex,
                        times_palace=times_palace,
                        )

Report generated successfully: Simulation Results and Analysis.html


'Simulation Results and Analysis.html'

In [53]:
if False:
    from qiskit_metal import designs, Dict
    x = designs.DesignPlanar({}, overwrite_enabled=True);
    gui = MetalGUI(feedline_design);
    gui.main_window.load_stylesheet_dark();
    gui.main_window.showMaximized();
    gui.autoscale();
    gui.qApp.exec_();
    #design.rebuild()
    #gui.rebuild()
    #gui.autoscale()
    #gui.show()  # Display the design in the GUI

    gui.main_window.force_close = True  
    gui.main_window.close();
    gui.qApp.quit()